# 用训练得到的模型预测-视频

同济子豪兄 2023-7-13

## 进入mmsegmentation目录

In [1]:
import os
os.chdir('mmsegmentation')

## MMSegmentation官方摄像头预测

In [ ]:
!python3 demo/video_demo.py \
         data/video_watermelon_3.mov \
         Zihao-Configs/ZihaoDataset_PSPNet_Mobile_20230712.py \
         checkpoint/Zihao_PSPNet-Mobile.pth \
         --device cpu \
         --opacity 0.5 \
         --show

/Users/tommy/Downloads/0713西瓜语义分割/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/Users/tommy/Downloads/0713西瓜语义分割/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
Loads checkpoint by local backend from path: checkpoint/Zihao_PSPNet-Mobile.pth
07/13 09:15:05 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
2023-07-13 09:15:07.129 Python[2889:30825821] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/ly/mdyk3b_d1r39hm00y4s1yk1m0000gn/T

## OpenCV

### 导入工具包

In [15]:
import time
import numpy as np
from tqdm import tqdm
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import mmcv
from mmseg.apis import init_model, inference_model

### 载入训练好的模型

In [3]:
# 载入 config 配置文件
from mmengine import Config
cfg = Config.fromfile('Zihao-Configs/ZihaoDataset_PSPNet_Mobile_20230712.py')

In [4]:
# 载入checkpoint权重文件
checkpoint_path = 'checkpoint/Zihao_PSPNet-Mobile.pth'

In [5]:
device = 'cpu'
# device = 'cuda:0'

model = init_model(cfg, checkpoint_path, device)

Loads checkpoint by local backend from path: checkpoint/Zihao_PSPNet-Mobile.pth


/Users/tommy/Downloads/0713西瓜语义分割/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/Users/tommy/Downloads/0713西瓜语义分割/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


### 各类别的配色方案（BGR）

In [6]:
palette = [
    ['background', [127,127,127]],
    ['red', [0,0,200]],
    ['green', [0,200,0]],
    ['white', [144,238,144]],
    ['seed-black', [30,30,30]],
    ['seed-white', [8,189,251]]
]

In [7]:
palette_dict = {}
for idx, each in enumerate(palette):
    palette_dict[idx] = each[1]

In [8]:
palette_dict

{0: [127, 127, 127],
 1: [0, 0, 200],
 2: [0, 200, 0],
 3: [144, 238, 144],
 4: [30, 30, 30],
 5: [8, 189, 251]}

### 逐帧处理函数

In [9]:
opacity = 0.3 # 透明度，越大越接近原图

In [10]:
def process_frame(img_bgr):
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    # 语义分割预测
    result = inference_model(model, img_bgr)
    pred_mask = result.pred_sem_seg.data[0].cpu().numpy()
    
    # 将预测的整数ID，映射为对应类别的颜色
    pred_mask_bgr = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3))
    for idx in palette_dict.keys():
        pred_mask_bgr[np.where(pred_mask==idx)] = palette_dict[idx]
    pred_mask_bgr = pred_mask_bgr.astype('uint8')
    
    # 将语义分割预测图和原图叠加显示
    img_bgr = cv2.addWeighted(img_bgr, opacity, pred_mask_bgr, 1-opacity, 0)

    return img_bgr

### 视频逐帧处理

In [11]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义process_frame函数即可
# 同济子豪兄 2021-7-10

def generate_video(input_path='videos/robot.mp4'):
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except:
                    print('报错！', error)
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

In [17]:
generate_video(input_path='data/video_watermelon_3.mov')

视频开始处理 data/video_watermelon_3.mov
视频总帧数为 6


100%|██████████| 5/5 [00:39<00:00,  7.98s/it]

视频已保存 out-video_watermelon_3.mov
